In [1]:
%load_ext autoreload
%autoreload 2

MONTAGE_SIZE=[4, 32]
AUGMENT_IMG=True
CROP=True
FILTERS=8

VIRTUAL_BATCH_SIZE=1
GPU_BATCH_SIZE=256
SGDR_CYCLE_LENGTH=1
SGDR_MULT_FACTOR=2.
SGDR_LR_DECAY=1.0
TARGET_SIZE=768
VALID_IMG_COUNT=32
NUM_EPOCHS=7
MINIBATCH_SIZE=5000
SGDR_MIN_LEARNING_RATE=1e-6
SGDR_MAX_LEARNING_RATE=1e-4
PATIENCE=128
model_load=None
MODEL_NAME='SGDR_separable_8'
BETA=5.
KAPPA=1.
TTA = False

In [2]:
import os
import numpy as np # linear algebra
np.random.seed(1)
ship_dir = '../data2/v2/'
train_image_dir = os.path.join(ship_dir, 'train')
test_image_dir = os.path.join(ship_dir, 'test')

# disable GPU usage while training
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
# # Generate Validation Data
from airbus_train_test import AirbusTrainValGen
from img_preprocess import tta_batch, tta_batch_reverse

train_val_gen = AirbusTrainValGen(train_image_dir,
                                  batch_size=GPU_BATCH_SIZE,
                                  valid_batch_size=GPU_BATCH_SIZE,
                                  target_size=TARGET_SIZE,
                                  augment=AUGMENT_IMG,
                                  crop=CROP,
                                  noship_detector=False,
                                  use_loss_weights=False)

Using TensorFlow backend.


## Build a Model (U-Net)

In [4]:
from model import get_resnet34
from optimizers import AdamAccumulate
from keras.optimizers import SGD
from losses import *

gpu_batches_to_accum = int(VIRTUAL_BATCH_SIZE / GPU_BATCH_SIZE)
seg_model = get_resnet34(input_shape=(TARGET_SIZE, TARGET_SIZE, 3))
seg_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 768, 768, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 768, 768, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 774, 774, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 384, 384, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn0 (Batch

In [5]:
t_x, t_y = next(train_val_gen.valid_gen)
img_renormalized = t_x + 0.5
if TTA:
    tta_x = tta_batch(t_x)

In [6]:
#seg_model.save('768_768_new_base_fix_5_best_loss_model.h5')

In [7]:
#tta_x = tta_batch(t_x)

In [8]:
seg_model.load_weights('checkpoints/768_768_resnet34_ohem_dice_best_loss_weights.h5')

if TTA:
    first_segs = [seg_model.predict(i, batch_size=1) for i in tta_x]
else:
    first_segs = seg_model.predict(t_x, batch_size=1)

In [9]:
if TTA:
    aligned_segs = np.stack(tta_batch_reverse(first_segs), 3)
    avg_seg = np.sum(aligned_segs, axis=3, keepdims=True)
    ratio = 4/4
    num_augs = aligned_segs.shape[3]
    threshold = 0.50

    comparison = threshold * ratio * num_augs

    first_pred = np.greater(avg_seg, comparison).astype('float32')
else:
    first_pred = np.greater(first_segs, 0.5).astype('float32')


In [10]:
import rle
from rle import instance_segment_mask, instance_segment_box
from img_preprocess import post_process_prediction
import cv2

pp_pred = np.stack( [post_process_prediction(i, structure_size=3, iterations=1) for i in first_pred[0:,...]], axis=0)
total_f2 = 0
total_f2_box = 0
total_pixel_ratio = 0

box_masks = []
for gt, pred in zip(t_y[0:,...], pp_pred[0:,...]):
    total_pixel_ratio += gt.sum()/gt.size
    
    ships_gt = instance_segment_mask(gt, smallest_size=0)
    ships_pred = instance_segment_mask(pred, smallest_size=50)
    
    '''
    boxes_pred = instance_segment_box(pred)
    img_box_masks = np.zeros((TARGET_SIZE, TARGET_SIZE, 1), dtype='uint8')
    for i in boxes_pred:
        img_box_masks = np.maximum(img_box_masks, i)
    box_masks += [np.concatenate([img_box_masks, np.zeros((TARGET_SIZE, TARGET_SIZE, 2))], axis=2)]
    '''
    
            
    f2_score = f2(ships_gt, ships_pred)
    #f2_score_box = f2(ships_gt, boxes_pred)
    total_f2 += f2_score
    #total_f2_box += f2_score_box
    #print('f2 score: {}'.format(f2_score))

print('final_f2: {}'.format(total_f2/GPU_BATCH_SIZE))
#print('final_f2_box: {}'.format(total_f2_box/GPU_BATCH_SIZE))
print('valid pixel ratio: {}'.format(total_pixel_ratio / GPU_BATCH_SIZE))

intersection = np.sum(t_y.flatten() * first_pred.flatten())
union = np.sum(t_y.flatten()) + np.sum(first_pred.flatten()) - intersection
print('iou: {}'.format(intersection / union))

#bbox_img = np.stack(box_masks, axis=0)

final_f2: 0.46587514720953466
valid pixel ratio: 0.01033304135004679
iou: 0.80078348125357


In [ ]:
from montage import *
from img_preprocess import *
from rle import *

mont_pred = pp_pred[0:,...,0]
    
montage(img_renormalized, MONTAGE_SIZE)
montage(t_y[:,:,:,0], MONTAGE_SIZE)
montage(mont_pred, MONTAGE_SIZE)
#montage_with_mask(bbox_img, mont_pred, MONTAGE_SIZE)
#montage_with_mask(bbox_img, t_y[:,:,:,0], MONTAGE_SIZE, img_size=10)
montage_with_mask(img_renormalized, mont_pred, MONTAGE_SIZE)